In [1]:
import random
import numpy as np

def read_file(file_path: str) -> [str]:
    with open(file_path, 'r') as f:
        texts = f.read().split('\n')
    return texts

def msra_preprocessing(split_rate: float = 0.8,
                       ignore_exist: bool = False) -> None:
    path_train = 'data/MSRA/train.txt'
    path_test = 'data/MSRA/test.txt'

    print("正在对MSRA数据进行预处理......")
    path_train1 = 'data/MSRA/train1.txt'
    texts = read_file(path_train1)
    random.shuffle(texts)
    
    split_index = int(len(texts) * split_rate)
    train_texts = texts[:split_index]
    test_texts = texts[split_index:]

    test_ = text_map(test_texts)
    train_ = text_map(train_texts)

    with open(path_train, 'w') as f:
        f.write("".join(train_))
    with open(path_test, 'w') as f:
        f.write("".join(test_))
    print("MSRA数据进行预处理完成 ---- OK!")

unk_flag = '[UNK]'
pad_flag = '[PAD]'
cls_flag = '[CLS]'
sep_flag = '[SEP]'

# 获取 word to index 词典
def get_w2i(vocab_path='data/vocab/vocab.txt'):
    w2i = {}
    with open(vocab_path, 'r') as f:
        while True:
            text = f.readline()
            if not text:
                break
            text = text.strip()
            if text and len(text) > 0:
                w2i[text] = len(w2i) + 1
    return w2i

# 文本映射
def text_map(texts: [str]) -> [str]:
    """
    文本映射处理
    处理好的数据格式:
       ['需 O'
        '要 O'
        '大 B-ORG'
        '连 I-ORG'
        '海 I-ORG'
        '富 I-ORG'
        '集 I-ORG'
        '团 I-ORG']

    :param texts:  例如 中/B_nt 共/M_nt 中/M_nt 央/E_nt 总/O  的文本
    :return: [str] 处理好的数据
    """
    mapping = {
        'nr': 'PER',
        'ns': 'LOC',
        'nt': 'ORG'
    }
    deal_texts = []
    for line in texts:
        sub_line = str(line).split(' ')
        for item in sub_line:
            item_list = str(item).split('/')
            if len(item_list) == 2:
                a = item_list[0]
                b = item_list[1]
                if b in mapping:
                    flag = mapping[b]
                    for i, char in enumerate(a):
                        if i == 0:
                            deal_texts.append(f"{char} B-{flag}\n")
                        else:
                            deal_texts.append(f"{char} I-{flag}\n")
                else:
                    for char in a:
                        deal_texts.append(f"{char} O\n")
        deal_texts.append('\n')
    return deal_texts

# 获取 tag to index 词典
def get_tag2index():
    return {"O": 0,
            "B-PER": 1, "I-PER": 2,
            "B-LOC": 3, "I-LOC": 4,
            "B-ORG": 5, "I-ORG": 6
            }
class DataProcess(object):
    def __init__(self,
                 max_len=100,
                 ):
        """
        数据处理
        :param max_len: 句子最长的长度，默认为保留100
        :param data_type: 数据类型，当前支持四种数据类型
        """
        self.w2i = get_w2i()  # word to index
        self.tag2index = get_tag2index()  # tag to index
        self.vocab_size = len(self.w2i)
        self.tag_size = len(self.tag2index)
        self.unk_flag = unk_flag
        self.pad_flag = pad_flag
        self.max_len = max_len

        self.unk_index = self.w2i.get(unk_flag, 101)
        self.pad_index = self.w2i.get(pad_flag, 1)
        self.cls_index = self.w2i.get(cls_flag, 102)
        self.sep_index = self.w2i.get(sep_flag, 103)
        msra_preprocessing()

    def get_data(self, one_hot: bool = True) -> ([], [], [], []):
        """
        获取数据，包括训练、测试数据中的数据和标签
        :param one_hot:
        :return:
        """
        # 拼接地址
        path_train = 'data/MSRA/train.txt'
        path_test = 'data/MSRA/test.txt'
        # 读取数据
        train_data, train_label = self.__text_to_indexs(path_train)
        test_data, test_label = self.__text_to_indexs(path_test)

        # 进行 one-hot处理
        if one_hot:
            def label_to_one_hot(index: []) -> []:
                data = []
                for line in index:
                    data_line = []
                    for i, index in enumerate(line):
                        line_line = [0]*self.tag_size
                        line_line[index] = 1
                        data_line.append(line_line)
                    data.append(data_line)
                return np.array(data)
            train_label = label_to_one_hot(index=train_label)
            test_label = label_to_one_hot(index=test_label)
        else:
            train_label = np.expand_dims(train_label, 2)
            test_label = np.expand_dims(test_label, 2)
        return train_data, train_label, test_data, test_label

    def num2tag(self):
        return dict(zip(self.tag2index.values(), self.tag2index.keys()))

    def i2w(self):
        return dict(zip(self.w2i.values(), self.w2i.keys()))

    # texts 转化为 index序列
    def __text_to_indexs(self, file_path: str) -> ([], []):
        data, label = [], []
        with open(file_path, 'r') as f:
            line_data,  line_label = [], []
            for line in f:
                if line != '\n':
                    w, t = line.split()
                    char_index = self.w2i.get(w, self.w2i[self.unk_flag])
                    tag_index = self.tag2index.get(t, 0)
                    line_data.append(char_index)
                    line_label.append(tag_index)
                else:
                    if len(line_data) < self.max_len:
                        pad_num = self.max_len - len(line_data)
                        line_data = [self.pad_index]*pad_num + line_data
                        line_label = [0]*pad_num + line_label
                    else:
                        line_data = line_data[:self.max_len]
                        line_label = line_label[:self.max_len]
                    data.append(line_data)
                    label.append(line_label)
                    line_data, line_label = [], []
        return np.array(data), np.array(label)

In [2]:
dp = DataProcess()
train_data, train_label, test_data, test_label = dp.get_data(one_hot=True)

正在对MSRA数据进行预处理......
MSRA数据进行预处理完成 ---- OK!


In [3]:
train_data.shape,train_label.shape

((37092, 100), (37092, 100, 7))

In [4]:
from keras.models import Model
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input
from keras_contrib.layers import CRF

class BILSTMCRF():
    def __init__(self,
                 vocab_size: int,
                 n_class: int,
                 embedding_dim: int = 128,
                 rnn_units: int = 128,
                 drop_rate: float = 0.5,
                 ):
        self.vocab_size = vocab_size
        self.n_class = n_class
        self.embedding_dim = embedding_dim
        self.rnn_units = rnn_units
        self.drop_rate = drop_rate

    def create_model(self):
        inputs = Input(shape=(None,))
        x = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim)(inputs)
        x = Bidirectional(LSTM(units=self.rnn_units, return_sequences=True))(x)
        x = Dropout(self.drop_rate)(x)
        x = Dense(self.n_class)(x)
        self.crf = CRF(self.n_class, sparse_target=False)
        x = self.crf(x)
        self.model = Model(inputs=inputs, outputs=x)
        self.model.summary()
        self.compile()
        return self.model

    def compile(self):
        self.model.compile('adam',
                           loss=self.crf.loss_function,
                           metrics=[self.crf.accuracy])

Using TensorFlow backend.
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Library/Python/3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 

In [5]:
model = BILSTMCRF(dp.vocab_size, dp.tag_size)

In [6]:
dp.vocab_size

21127

In [7]:
dp.tag_size

7

In [8]:
model.create_model()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         2704256   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 7)           1799      
_________________________________________________________________
crf_1 (CRF)                  (None, None, 7)           119       
Total params: 2,969,342

/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Library/Python/3.7/site-packages/keras_contrib-2.0.8-py3.7.egg/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [12]:
model.model.fit(train_data, train_label, batch_size=128, epochs=10,
              validation_data=[test_data, test_label])


Train on 37092 samples, validate on 9273 samples
Epoch 1/10
37092/37092 [==============================] - 129s 3ms/step - loss: 0.2157 - crf_viterbi_accuracy: 0.9386 - val_loss: 0.1046 - val_crf_viterbi_accuracy: 0.9649
Epoch 2/10
37092/37092 [==============================] - 130s 4ms/step - loss: 0.0769 - crf_viterbi_accuracy: 0.9746 - val_loss: 0.0586 - val_crf_viterbi_accuracy: 0.9795
Epoch 3/10
37092/37092 [==============================] - 128s 3ms/step - loss: 0.0513 - crf_viterbi_accuracy: 0.9820 - val_loss: 0.0454 - val_crf_viterbi_accuracy: 0.9834
Epoch 4/10
37092/37092 [==============================] - 132s 4ms/step - loss: 0.0408 - crf_viterbi_accuracy: 0.9850 - val_loss: 0.0388 - val_crf_viterbi_accuracy: 0.9853
Epoch 5/10
37092/37092 [==============================] - 136s 4ms/step - loss: 0.0339 - crf_viterbi_accuracy: 0.9872 - val_loss: 0.0333 - val_crf_viterbi_accuracy: 0.9868
Epoch 6/10
37092/37092 [==============================] - 140s 4ms/step - loss: 0.0285 - c